# Generating melody *and* harmony using LSTMs

Again, using data from the Weimar Jazz Database.

## Approach #1: Add underlying harmony directly to note events

Functions here will eventually be migrated to the `jazzaiexperiments` module.

Import everything we need:

In [1]:
import mido
import music21
import pandas as pd

from jazzaiexperiments import *  # For easier testing/debugging

Using TensorFlow backend.


### Add chord info (from database) to note events

In [2]:
# Testing + building jazzaiexperiments functions
# Based on lstm_train_on_midi_input()

# Set variables (that will be passed in as arguments)
tune_name = "ColemanHawkins_BodyAndSoul_FINAL"
midi_data_dir = "../data/midi/quantized/"
checkpoints_data_dir = "../data/models/"
seq_length = 10
num_epochs = 1

# Create note events
input_filepath = midi_construct_input_filepath(tune_name, midi_data_dir)
midi_track = midi_load_melody_from_file(input_filepath)
note_pairs = midi_extract_note_pairs(midi_track)
note_pairs = midi_normalize_velocities(note_pairs, interval=10)
note_events = midi_create_note_events(note_pairs)
print("Created note events from {}".format(input_filepath))

# Format note data to feed into network
note_set = midi_create_note_set(note_events)
seqs_input, seqs_output = midi_split_subsequences(note_events,
                                                  seq_length=seq_length)
num_seqs = len(seqs_input)
seq_length = len(seqs_input[0])
num_unique_notes = len(note_set)
x, y = midi_format_for_lstm(seqs_input, seqs_output,
                            num_seqs=num_seqs,
                            seq_length=seq_length,
                            num_unique_notes=num_unique_notes)
print("Formatted note data ({} seqs of length {}, "
      "{} unique notes)".format(num_seqs, seq_length, num_unique_notes))

Created note events from ../data/midi/quantized/ColemanHawkins_BodyAndSoul_FINAL.mid
Formatted note data (625 seqs of length 10, 365 unique notes)


In [3]:
def midi_note_event_to_dict(note):
    note_events_keys = midi_get_note_event_keys()
    return dict((note_events_keys[i], note[i]) for i,_ in enumerate(note))

midi_note_event_to_dict(note_events[0])

{'noteoff_time': 16170,
 'noteon_pitch': 51,
 'noteon_time': 76230,
 'noteon_velocity': 100}

In [4]:
[msg for msg in midi_track if "note" not in msg.type]

[<meta message track_name name='Melody' time=0>,
 <meta message set_tempo tempo=631578 time=0>,
 <meta message key_signature key='Db' time=0>,
 <meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>,
 <meta message end_of_track time=0>]

In [5]:
midi_track[5]

<message note_off channel=0 note=51 velocity=100 time=16170>

In [6]:
mf = mido.MidiFile(input_filepath)
mf.ticks_per_beat

27720

In [7]:
# Calculate times of note onsets (in seconds)
def calculate_note_times_seconds(input_filepath):
    midi_file = mido.MidiFile(input_filepath)
    midi_track = midi_load_melody_from_file(input_filepath)
    tempo = midi_track[1].tempo
    # ppq = midi_track[3].clocks_per_click
    # n32 = midi_track[3].notated_32nd_notes_per_beat
    ppq = midi_file.ticks_per_beat
    note_times = [mido.tick2second(msg.time, ppq, tempo) for msg in midi_track if "note" in msg.type]
    note_times_summed = []
    for i, t in enumerate(note_times):
        note_times_summed.append(sum(note_times[:i]) + t)
    return note_times_summed

calculate_note_times_seconds(input_filepath)[:10]

[1.7368395,
 2.10526,
 2.10526,
 2.315786,
 2.315786,
 3.3157845000000004,
 3.3157845000000004,
 3.473679,
 3.473679,
 3.6315735]

Actually, let's get the current chord directly from the database. Here I've exported the **beats** and **melody** from the database using a simple:

    SELECT * FROM beats WHERE melid=96
    
and

    SELECT * FROM melody WHERE melid=96
    
with 96 being the melody ID for Coleman Hawkins - Body and Soul

In [8]:
len(note_events)

635

In [9]:
def db_read_file(filepath):
    data = pd.read_csv(filepath)
    return data

db_read_file("../data/db/ColemanHawkins_BodyAndSoul_FINAL_beats.csv").head()

,beatid,melid,onset,bar,beat,signature,chord,form,bass_pitch,chorus_id
0,24777,96,8.440748,0,3,NaN,NC,I1,51,0
1,24778,96,9.035805,0,4,NaN,NaN,NaN,51,0
2,24779,96,9.807823,1,1,4/4,Eb-,A1,34,1
3,24780,96,10.452993,1,2,NaN,NaN,NaN,42,1
4,24781,96,11.128458,1,3,NaN,D+7,NaN,41,1


In [10]:
db_read_file("../data/db/ColemanHawkins_BodyAndSoul_FINAL_melody.csv").head()

,eventid,melid,onset,pitch,duration,period,division,bar,beat,tatum,...,f0_mod,loud_max,loud_med,loud_sd,loud_relpos,loud_cent,loud_s2b,f0_range,f0_freq_hz,f0_med_dev
0,41757,96,8.939683,51.0,0.278639,4,4,0,3,4,...,NaN,0.552351,62.516859,2.913798,0.240000,0.368501,1.181904,21.915053,7.412672,-11.070815
1,41758,96,9.218322,51.0,0.290249,4,3,0,4,2,...,NaN,0.370392,59.980021,1.694662,0.481481,0.443516,1.180483,38.993741,4.401113,-17.066446
2,41759,96,9.508571,51.0,0.371519,4,3,0,4,3,...,NaN,0.432227,58.906740,3.292181,0.441176,0.450475,1.104257,87.635138,4.641182,-32.029418
3,41760,96,10.582494,51.0,0.191565,4,4,1,2,2,...,NaN,0.720836,64.238410,3.861087,0.058824,0.400048,1.193654,88.286175,5.556956,1.528574
4,41761,96,10.808889,53.0,0.145102,4,4,1,2,3,...,NaN,0.530749,61.655251,2.110308,0.153846,0.489615,1.168982,101.227998,7.177734,12.858254


In [11]:
data_beats.dropna?

Object `data_beats.dropna` not found.


In [12]:
def db_get_harmony_for_melody(beats_filepath, melody_filepath):
    data_beats = db_read_file(beats_filepath)
    data_melody = db_read_file(melody_filepath)
    chords = []
    for i,melevt in data_melody.iterrows():
        # Get beats that came before current melody event (i.e. note)
        beats = data_beats.dropna(subset=["chord"])[data_beats.onset < melevt.onset]
        if len(beats) < 1:
            chords.append("NC")
            continue

        # Get most recent chord
        most_recent_beat = beats.iloc[-1:]
        #print("{}: {}".format(i, len(beats)))
        #print(most_recent_beat.chord)
        chord = most_recent_beat.chord
        chord = chord[chord.keys()[0]] if len(chord.keys()) > 0 else "NC"
        chords.append(chord)
    return chords
        
beats_filepath = "../data/db/ColemanHawkins_BodyAndSoul_FINAL_beats.csv"
melody_filepath = "../data/db/ColemanHawkins_BodyAndSoul_FINAL_melody.csv"
chords = db_get_harmony_for_melody(beats_filepath, melody_filepath)
chords[:10]

/Users/usdivad/.pyenv/versions/3.5.1/lib/python3.5/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


['NC', 'NC', 'NC', 'Eb-', 'Eb-', 'Eb-', 'D+7', 'D+7', 'D+7', 'D+7']

In [13]:
# This will be a variant of midi_create_note_events()
def midi_create_note_events_harmony(note_pairs, chords):
    note_events = [(note_on.note, note_on.velocity,
                    note_on.time, note_off.time,
                    chords[i])
                   for i, (note_on, note_off) in enumerate(note_pairs)]
    return note_events

note_events = midi_create_note_events_harmony(note_pairs, chords)
note_events[:10]

[(51, 100, 76230, 16170, 'NC'),
 (51, 90, 0, 9240, 'NC'),
 (51, 90, 0, 43890, 'NC'),
 (51, 100, 0, 6930, 'Eb-'),
 (53, 90, 0, 6930, 'Eb-'),
 (54, 110, 0, 6930, 'Eb-'),
 (53, 100, 0, 4620, 'D+7'),
 (54, 100, 0, 4620, 'D+7'),
 (53, 90, 0, 39270, 'D+7'),
 (51, 90, 0, 13860, 'D+7')]

In [14]:
# We already did this in db_get_harmony_for_melody() and midi_create_note_events_harmony()
# def midi_add_harmony_to_notes(note_events):
#     # TODO
#     # So we'll need the note events but also the chordal info. From a CSV export from the database itself??
#     pass

Now let's put it all back into our function:

In [15]:
# def lstm_train_with_harmony():

def lstm_train_on_midi_input(tune_name,
                             midi_data_dir="../data/midi/",
                             checkpoints_data_dir="../data/models/",
                             input_filepath=None,
                             weights_filepath=None,
                             db_beats_filepath=None,
                             db_melody_filepath=None,
                             seq_length=10,
                             num_epochs=100,
                             mode="single_melody"):
    """Build and train an LSTM from an input MIDI file."""
    # Load MIDI file
    if input_filepath is None:
        input_filepath = midi_construct_input_filepath(tune_name,
                                                       midi_data_dir)
    midi_track = midi_load_melody_from_file(input_filepath)
    note_pairs = midi_extract_note_pairs(midi_track)
    note_pairs = midi_normalize_velocities(note_pairs, interval=10)
    
    # Get harmony data
    chords = db_get_harmony_for_melody(db_beats_filepath, db_melody_filepath)
    
    # Create note events
    note_events = []
    if mode == "single_melody":
        note_events = midi_create_note_events(note_pairs)
    elif mode == "single_melody_harmony":
        note_events = midi_create_note_events_harmony(note_pairs, chords)
    print("Created {} note events from {} using mode {}".format(len(note_events), input_filepath, mode))

    # Format note data to feed into network
    note_set = midi_create_note_set(note_events)
    seqs_input, seqs_output = midi_split_subsequences(note_events,
                                                      seq_length=seq_length)
    num_seqs = len(seqs_input)
    seq_length = len(seqs_input[0])
    num_unique_notes = len(note_set)
    x, y = midi_format_for_lstm(seqs_input, seqs_output,
                                num_seqs=num_seqs,
                                seq_length=seq_length,
                                num_unique_notes=num_unique_notes)
    print("Formatted note data ({} seqs of length {}, "
          "{} unique notes)".format(num_seqs, seq_length, num_unique_notes))

    # Create LSTM
    model = lstm_create(x.shape, y.shape, num_units=256, dropout_rate=0.2)
    print("Created model")

    # Train LSTM, or load from weights
    if weights_filepath is None:
        callbacks = lstm_setup_callbacks(tune_name, checkpoints_data_dir)
        model = lstm_fit_model(model, x, y,
                               num_epochs=num_epochs,
                               batch_size=32,
                               callbacks=callbacks)
        print("Trained model over {} epochs".format(num_epochs))
    else:
        model = lstm_load_weights(model, weights_filepath)
        print("Loaded weights from {}".format(weights_filepath))

    return (model, note_events, input_filepath)
    
# # Set variables (that will be passed in as arguments)
# tune_name = "ColemanHawkins_BodyAndSoul_FINAL"
# midi_data_dir = "../data/midi/quantized/"
# checkpoints_data_dir = "../data/models/"
# input_filepath = None
# beats_filepath = "../data/db/ColemanHawkins_BodyAndSoul_FINAL_beats.csv"
# melody_filepath = "../data/db/ColemanHawkins_BodyAndSoul_FINAL_melody.csv"
# weights_filepath = None
# seq_length = 10
# num_epochs = 1
# mode = "single_melody_harmony"
# #    

trained = lstm_train_on_midi_input(tune_name="ColemanHawkins_BodyAndSoul_FINAL",
                                   midi_data_dir="../data/midi/quantized/",
                                   db_beats_filepath="../data/db/ColemanHawkins_BodyAndSoul_FINAL_beats.csv",
                                   db_melody_filepath="../data/db/ColemanHawkins_BodyAndSoul_FINAL_melody.csv",
                                   seq_length=10,
                                   num_epochs=1,
                                   mode="single_melody_harmony")
model, note_events, input_filepath = trained

/Users/usdivad/.pyenv/versions/3.5.1/lib/python3.5/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


Created 635 note events from ../data/midi/quantized/ColemanHawkins_BodyAndSoul_FINAL.mid using mode single_melody_harmony
Formatted note data (625 seqs of length 10, 581 unique notes)
Created model
Epoch 1/1
625/625 [==============================] - 4s - loss: 6.3754     
Trained model over 1 epochs


In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10, 256)           264192    
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 256)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 581)               149317    
Total params: 938,821
Trainable params: 938,821
Non-trainable params: 0
_________________________________________________________________


In [17]:
note_events[:10]

[(51, 100, 76230, 16170, 'NC'),
 (51, 90, 0, 9240, 'NC'),
 (51, 90, 0, 43890, 'NC'),
 (51, 100, 0, 6930, 'Eb-'),
 (53, 90, 0, 6930, 'Eb-'),
 (54, 110, 0, 6930, 'Eb-'),
 (53, 100, 0, 4620, 'D+7'),
 (54, 100, 0, 4620, 'D+7'),
 (53, 90, 0, 39270, 'D+7'),
 (51, 90, 0, 13860, 'D+7')]

### Convert chord names to MIDI pitch information

We need some way to convert e.g. "Eb-" to useful note data. `music21` looks handy for that (on second thought, perhaps I should rewrite MIDI functions to use `music21` as well):

In [18]:
[pitch.midi for pitch in music21.harmony.ChordSymbol("E-m").pitches]

[51, 54, 58]

Alright, so we have a bit of conversion to do to get it in a nice format for `music21.harmony.ChordSymbol` (e.g. from "Eb-" to "E-m"). This takes some string wrangling:

In [19]:
def db_chord_to_music21_chord(chord):
    new_chord = chord
    
    # Use "m" instead of "-" to indicate minor chords
    new_chord = new_chord.replace("-", "m")
    
    # Use "maj" instead of "j" to indicate maj7 chords
    new_chord = new_chord.replace("j", "maj")
    
    # Use "sus4add7" instead of "sus7" for suspended 7th chords
    # TODO: Get it to add a dominant 7th ("sus4add7" adds a major 7th)
    #       (probably could use music21.harmony.addNewChordSymbol())
    new_chord = new_chord.replace("sus7", "sus4")
    
    # According to music21 docs: "if a root or bass is flat, the ‘-‘ must be used, and NOT ‘b’.
    # However, alterations and chord abbreviations are specified normally with the ‘b’ and ‘#’ signs."
    if len(new_chord) > 1 and new_chord[1] == "b":
        new_chord_arr = list(new_chord)
        new_chord_arr[1] = "-"
        new_chord = "".join(new_chord_arr)
    
    # Convert e.g. "B-79b" to "B-7b9"
    if len(new_chord) > 0 and new_chord[-1] == "b":
        new_chord_arr = list(new_chord)
        if len(new_chord_arr) > 2:
            new_chord_arr[-1] = new_chord_arr[-2]
            new_chord_arr[-2] = "b"
        new_chord = "".join(new_chord_arr)
        
    return new_chord

chord_set = set(chords)
chord_set_converted = set([db_chord_to_music21_chord(chord) for chord in chord_set])
chord_set_converted
[music21.harmony.ChordSymbol(chord) for chord in chord_set_converted if chord != "NC"][:10]

# for chord in chord_set_converted:
#     print(chord)
#     if chord != "NC":
#         print(music21.harmony.ChordSymbol(chord))

[<music21.harmony.ChordSymbol F#>,
 <music21.harmony.ChordSymbol E-m7>,
 <music21.harmony.ChordSymbol Gm7>,
 <music21.harmony.ChordSymbol E-m>,
 <music21.harmony.ChordSymbol F#m7>,
 <music21.harmony.ChordSymbol Em7>,
 <music21.harmony.ChordSymbol E-o>,
 <music21.harmony.ChordSymbol B-sus4>,
 <music21.harmony.ChordSymbol Fm7>,
 <music21.harmony.ChordSymbol D+7>]

And then to MIDI pitches:

In [20]:
def db_chord_to_midi_pitches(chord):  # TODO: Add optional octave argument
    if chord == "NC":
        return []
    chord_name = db_chord_to_music21_chord(chord)
    chord_symbol = music21.harmony.ChordSymbol(chord_name)
    return [pitch.midi for pitch in chord_symbol.pitches]

[db_chord_to_midi_pitches(chord) for chord in chord_set][:10]

[[54, 58, 61],
 [46, 50, 53, 56],
 [46, 49, 53, 56],
 [42, 46, 49, 52],
 [42, 45, 49, 52],
 [46, 50, 53, 56, 59],
 [41, 44, 48, 51],
 [44, 48, 51, 54],
 [51, 54, 58],
 [51, 54, 58, 61]]

OK I think we're ready to start generating some notes!

### Generate notes

We'll have to rewrite the function that retrieves keys for note events, so that it will handle chord info:

In [21]:
def midi_get_note_event_keys(mode="single_melody"):
    """Get keys for note events."""
    if mode == "single_melody":
        # Don't use note off velocity to shrink possibilities,
        # and don't use note off pitch because it's the same as note on pitch
        return ("noteon_pitch", "noteon_velocity",
                "noteon_time", "noteoff_time")
    elif mode == "single_melody_harmony":
        return ("noteon_pitch", "noteon_velocity",
                "noteon_time", "noteoff_time",
                "chord")
    return ()

midi_get_note_event_keys(mode="single_melody_harmony")

('noteon_pitch', 'noteon_velocity', 'noteon_time', 'noteoff_time', 'chord')

All we have to do to `lstm_generate_midi_output` is pass the mode into `midi_write_file`:

In [22]:
def lstm_generate_midi_output(model, note_events,
                              mode="single_melody",
                              num_notes_to_generate=100,
                              batch_size=32,
                              random_seed=False,
                              add_seed_to_output=False,
                              output_filepath=None,
                              midi_source_filepath=None,
                              tune_name="output",
                              data_dir="../data/output/"):
    """Generate note output, given a trained model."""
    # Construct input sequence
    seq_in = lstm_construct_input_seq(note_events, model.input_shape[1],
                                      random_seed=random_seed)
    print("Constructed input sequence: {}".format(seq_in))

    # Generate the notes!
    num_notes = num_notes_to_generate
    notes_out = lstm_generate_notes(model, note_events, seq_in,
                                    num_notes_to_generate=num_notes,
                                    batch_size=batch_size,
                                    add_seed_to_output=add_seed_to_output)
    print("Generated {} notes".format(num_notes))

    # Write output to MIDI file
    if output_filepath is None:
        output_filepath = midi_construct_output_filepath(tune_name, data_dir)
    midi_write_file(notes_out, output_filepath,
                    mode=mode,
                    midi_source_filepath=midi_source_filepath)
    print("Wrote to MIDI file at {}".format(output_filepath))
    return notes_out

Similarly, get `midi_write_file` to pass the mode into `midi_get_note_event_keys`, and also write the chord information to file:

In [30]:
def midi_write_file(note_events, output_filepath,
                    mode="single_melody",
                    midi_source_filepath=None):
    """Write note events to MIDI file.

    Convert the sequence of note tuples into a sequence of MIDI notes,
    and then write to MIDI file.
    """
    # Create MIDI file and track
    midi_file_out = mido.MidiFile()
    midi_track_out = mido.MidiTrack()
    midi_file_out.tracks.append(midi_track_out)

    # Append "headers" (track name, tempo, key, time signature)
    if midi_source_filepath is not None:
        midi_track = midi_load_melody_from_file(midi_source_filepath)
        for message in midi_track[:4]:
            midi_track_out.append(message)
    else:
        pass

    # Add notes
    prev_time = 0
    note_events_keys = midi_get_note_event_keys(mode=mode)

    # Note times get all bunched together, so we stretch them out
    # a little bit manually here...
    # TODO: Revisit this to make it more robust
    time_multiplier = 2  # Art Pepper - Anthropology
    time_multiplier = 0.02  # Coleman Hawkins - Body and Soul
    
    # Harmony (chord) settings
    prev_chord = "NC"
    chord_velocity = 64

    for note in note_events:
        # Note on/off pairs
        note = dict((note_events_keys[i], note[i]) for i, _ in enumerate(note))
        noteon_time = int(note["noteon_time"] * time_multiplier)
        noteoff_time = int(note["noteoff_time"] * time_multiplier)
        curr_time_noteon = prev_time + noteon_time
        curr_time_noteoff = prev_time + noteoff_time
        # prev_time = curr_time_noteoff
        message_noteon = mido.Message("note_on",
                                      note=note["noteon_pitch"],
                                      velocity=note["noteon_velocity"],
                                      time=curr_time_noteon)
        message_noteoff = mido.Message("note_off",
                                       note=note["noteon_pitch"],
                                       velocity=note["noteon_velocity"],
                                       time=curr_time_noteoff)
        midi_track_out.append(message_noteon)
        midi_track_out.append(message_noteoff)
        
        # Write harmony (chords) as well
        if mode == "single_melody_harmony":
            curr_chord = note["chord"]
            if curr_chord != prev_chord:
                curr_pitches = db_chord_to_midi_pitches(curr_chord)
                prev_pitches = db_chord_to_midi_pitches(prev_chord)
                
                # Add note ons for current chord
                for pitch in curr_pitches:
                    message = mido.Message("note_on",
                                           note=pitch,
                                           velocity=chord_velocity,
                                           time=curr_time_noteon)
                    midi_track_out.append(message)

                # Add note offs for previous chord
                for pitch in prev_pitches:
                    message = mido.Message("note_off",
                                           note=pitch,
                                           velocity=chord_velocity,
                                           time=curr_time_noteon)
                    midi_track_out.append(message)
                
            prev_chord = curr_chord

    # Save file to disk
    midi_file_out.save(output_filepath)

    # for message in midi_track_out[4:20]:
    #     print(message)
    return output_filepath

### Putting it all together

Train the model:

In [24]:
trained = lstm_train_on_midi_input(tune_name="ColemanHawkins_BodyAndSoul_FINAL",
                                   mode="single_melody_harmony",
                                   midi_data_dir="../data/midi/quantized/",
                                   db_beats_filepath="../data/db/ColemanHawkins_BodyAndSoul_FINAL_beats.csv",
                                   db_melody_filepath="../data/db/ColemanHawkins_BodyAndSoul_FINAL_melody.csv",
                                   seq_length=10,
                                   num_epochs=1)
model, note_events, input_filepath = trained

/Users/usdivad/.pyenv/versions/3.5.1/lib/python3.5/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


Created 635 note events from ../data/midi/quantized/ColemanHawkins_BodyAndSoul_FINAL.mid using mode single_melody_harmony
Formatted note data (625 seqs of length 10, 581 unique notes)
Created model
Epoch 1/1
625/625 [==============================] - 4s - loss: 6.3755     
Trained model over 1 epochs


Use the model to generate some notes:

In [31]:
notes_out = lstm_generate_midi_output(model, note_events,
                                      mode="single_melody_harmony",
                                      num_notes_to_generate=100,
                                      random_seed=False,
                                      add_seed_to_output=True,
                                      tune_name=tune_name,
                                      midi_source_filepath=input_filepath,
                                      data_dir="../data/output")
notes_out[:20]

Constructed input sequence: [80, 70, 73, 77, 120, 184, 133, 155, 131, 72]
Generated 100 notes
Wrote to MIDI file at ../data/output/out_ColemanHawkins_BodyAndSoul_FINAL_20170703161631008511.mid


[(51, 100, 76230, 16170, 'NC'),
 (51, 90, 0, 9240, 'NC'),
 (51, 90, 0, 43890, 'NC'),
 (51, 100, 0, 6930, 'Eb-'),
 (53, 90, 0, 6930, 'Eb-'),
 (54, 110, 0, 6930, 'Eb-'),
 (53, 100, 0, 4620, 'D+7'),
 (54, 100, 0, 4620, 'D+7'),
 (53, 90, 0, 39270, 'D+7'),
 (51, 90, 0, 13860, 'D+7'),
 (60, 110, 0, 6930, 'Cj7'),
 (60, 110, 0, 6930, 'Cj7'),
 (60, 110, 0, 6930, 'Cj7'),
 (60, 110, 0, 6930, 'Cj7'),
 (61, 90, 0, 6930, 'Eb-7'),
 (61, 90, 0, 6930, 'Eb-7'),
 (61, 90, 0, 6930, 'Eb-7'),
 (61, 90, 0, 6930, 'Eb-7'),
 (61, 90, 0, 6930, 'Eb-7'),
 (61, 90, 0, 6930, 'Eb-7')]